**The problem of 2class classification using ktrain**

**Import libraries**

In [ ]:
!pip install -q ktrain

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text

In [ ]:
tf.__version__

'2.13.0'

**Import dataset**

In [ ]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

fatal: destination path 'IMDB-Movie-Reviews-Large-Dataset-50k' already exists and is not an empty directory.


In [ ]:
train = pd.read_excel('IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype = str)
test = pd.read_excel('IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype = str)

In [ ]:
train.sample(5)

,Reviews,Sentiment
23351,Monstrous mother-son-duo (Alice Krige and Bria...,pos
17829,I recently bought this movie and I do not regr...,pos
7582,"Haven't seen any of the Japanese Grudge-films,...",pos
7617,"I thought this was a great idea but, boy, was ...",neg
5507,This is one of the best comedy ever ! The writ...,pos


In [ ]:
print('train shape -', train.shape, '; test shape -', test.shape)

train shape - (25000, 2) ; test shape - (25000, 2)


**Splitting**

In [ ]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=train,
                                                                   text_column = 'Reviews',
                                                                   label_columns = 'Sentiment',
                                                                   val_df = test,
                                                                   maxlen = 500,
                                                                   preprocess_mode = 'bert')

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


**Model**

In [ ]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 500


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


done.


In [ ]:
learner = ktrain.get_learner(model = model,
                             train_data = (X_train, y_train),
                             val_data = (X_test, y_test),
                             batch_size = 6)

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('/content/drive/My Drive/bert')



begin training using onecycle policy with max lr of 2e-05...
   2/4167 [..............................] - ETA: 49:09:42 - loss: 0.9615 - accuracy: 0.5000

In [ ]:
sample_test = ['this movie was horrible, the plot was really boring. acting was okay',
               'the fild is really sucked. there is not plot and acting was bad',
               'what a beautiful movie. great plot. acting was good. will see it again']

In [ ]:
predictor.predict(sample_test)

In [ ]:
predictor.predict(data, return_proba=True)

#array([[0.99797565, 0.00202436],
 #      [0.99606663, 0.00393336],
   #    [0.00292433, 0.9970757 ]], dtype=float32)